

> A **derivative** captures the smallest absolute change of a function at a point. It is described by the tangent line, which locally mirrors the phenomenon and shows how it would change if it moved linearly in either direction. This requires the phenomenon to be functionally mappable to its surroundings. As the gap between points shrinks, the observed change converges to the true instantaneous rate of change. Expressing this process as a function allows us to obtain the rate of change at any point without redrawing tangents each time.
That's why the derivate is a function because it can give you the change in at any particular point's direction so we don't have to be drawing tangent lines for each points to be estimating it all the time.  
A **small change** refers to an absolute change in distance along the input axis. It does not necessarily attach to a strict fixed position, but instead represents the local stretch around the point under investigation. As this stretch becomes smaller, the change in the function’s output converges to the true instantaneous rate of change at that point.it’s about the size of the difference, not where the difference occurs.



In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Power rule

You're designing a rocket launch sequence. The rocket's height follows H(t) = -16t² + 200t + 50 feet at time t seconds.  
**Critical Questions:**

1. How fast is the rocket moving at any moment?
2. When does it reach maximum height?
3. When does it hit the ground?  

Equations:  
Height: H(t) = -16t² + 200t + 50  
Velocity: V(t) = H'(t) = -32t + 200  (derivative gives rate of change)  
Acceleration: A(t) = V'(t) = -32  (second derivative)


In [3]:
def rocket_launch_analysis():
  "rocket trajectory analysis"
  "Height: H(t) = -16t² + 200t + 50"
  "Velocity: V(t) = H'(t) = -32t + 200"
  "Acceleration: A(t) = V'(t) = -32"

  def height_function(t):
    return -16*t**(2) + 200*t + 50

  def velcoity_function(t):
    return -32*t + 200

  def acceleration_function(t):
    return -32

  def compute_derivatives_pytorch(timepoint):
    "Compute h,v,a at specific time"
    t = torch.tensor(timepoint, dtype=torch.float32, requires_grad = True) # track all operations on the tensor so we can later differentiate with respect to it.
    height = -16*t**(2) + 200*t + 50
    v = torch.autograd.grad(height, t, create_graph=True)[0]
    a = torch.autograd.grad(v, t, create_graph=True)[0]
    return height.item(), v.item(), a.item()
  test_times = [0,2.5, 5.0,6.25,10.0]
  print("Time | Height | Vel(Comp) |")
  print("-" * 50)

  for t in test_times:
    result = compute_derivatives_pytorch(t)
    print (result)

rocket_launch_analysis()

Time | Height | Vel(Comp) |
--------------------------------------------------
(50.0, 200.0, -32.0)
(450.0, 120.0, -32.0)
(650.0, 40.0, -32.0)
(675.0, 0.0, -32.0)
(450.0, -120.0, -32.0)


requires_grad=True → tells PyTorch to track computations for differentiation.

.backward() → computes derivative of a scalar w.r.t. inputs, and stores result in .grad.

torch.autograd.grad() → computes derivative and returns it directly (cleaner for math, higher derivatives).

create_graph=True → keeps the graph alive so you can differentiate again (second, third derivatives).

In [4]:
import jax
import jax.numpy as jnp

def jax_rocket_launch_analysis():

  #define function
  def height(t):
    return -16*t**(2) + 200*t + 50
  # create derivative functions
  v = jax.grad(height)
  a = jax.grad(v)
  test_points = jnp.array([0,2.5, 5.0,6.25,10.0])

  #vectorized computations:
  heights = jax.vmap(height)(test_points)
  velocities = jax.vmap(v)(test_points)
  accelerations = jax.vmap(a)(test_points)
  print("JAX Results:")
  print("Time | Height | Velocity | Acceleration")
  print("-" * 40)

  for i, t in enumerate(test_points):
    print(f"{t:.2f} | {heights[i]:.2f} | {velocities[i]:.2f} | {accelerations[i]:.2f}")

jax_rocket_launch_analysis()



JAX Results:
Time | Height | Velocity | Acceleration
----------------------------------------
0.00 | 50.00 | 200.00 | -32.00
2.50 | 450.00 | 120.00 | -32.00
5.00 | 650.00 | 40.00 | -32.00
6.25 | 675.00 | 0.00 | -32.00
10.00 | 450.00 | -120.00 | -32.00


for velocity = jax.vmap(velocity)(test_point)  
height is your function: def height(t): return -16*t**2 + 200*t + 50

test_points is a vector/array of time values.

jax.vmap(height) makes a new function that applies height to each element of the array in parallel.

Then (test_points) calls it → you get an array of heights at all those times.

PyTorch

You usually work with tensors as variables (torch.tensor(...)).

The function is written as tensor operations (H = -16*t**2 + ...).

Derivatives come out as tensors too (torch.autograd.grad(...)).

So both “inputs” and “outputs” of differentiation live in the tensor world.

JAX

You usually write pure Python functions (def f(x): return ...).

Then you wrap them with jax.grad(f) or jax.jacfwd(f) etc. to create new functions that compute derivatives.

When you call those derivative functions, they return values (often arrays) that act like normal Python variables.

The length l metres of a certain metal rod at temperature $θ^◦C$ is given by:  
 $l=1+0.00005θ+0.0000004θ^2$.  
Determine
 the rate of change of length, in $mm/^◦C$, when the
 temperature is (a) $100^◦C$ and (b) $400^◦C$.

In [5]:
# solving in pytorch:

def rod_length_analysis():
  "determining the length of a metal rod per change in temperature"
  "l = 1 + 0.00005*O +0.0000004*O**2"

  def compute_derivatives_pytorch(temp):
    # compute length at a specific temperature
    O = torch.tensor(temp, dtype=torch.float32, requires_grad=True) # create scalar variable tensor
    funtion = 1 + (0.00005*O) + (0.0000004) *(O**2) # scalar variable goes into the graph
    length = torch.autograd.grad(funtion, O, create_graph=True)[0] # function first, variable second, create graph should be true in case of anything
    return length.item()

  temp = [100, 400]

  for t in temp:
    result = compute_derivatives_pytorch(t)
    print(f"result for {t} degrees celsius: {result}")

rod_length_analysis()

result for 100 degrees celsius: 0.00013000000035390258
result for 400 degrees celsius: 0.0003699999942909926


In [6]:
def jax_rod_length_analysis():
  "rod analysis in jax"

  # does matter where i create as long as i create variable and intialise it after the variable i am good.
  def lenth_function(temp):
    return 1 + (0.00005*temp) + (0.0000004) *(temp**2)

  # creating derivative functions(auto complete tried to confuse me but remembered on my own)
  length = jax.grad(lenth_function) # python has this thing where ot creates functions but doesn't add the parenthesis when in some functions
  temp = jnp.array([100.0, 400.0])

  new_length = jax.vmap(length)(temp) # derivative function goes in here

  print("JAX Results:")
  print("Time | new_Length")
  print("-" * 40)

  for i, t in enumerate(temp):
    print(f"{t:.2f} | {new_length[i]:.7f}")  # c

jax_rod_length_analysis()


JAX Results:
Time | new_Length
----------------------------------------
100.00 | 0.0001300
400.00 | 0.0003700


**Product and Quotient rules** - Complex system analysis

An AC circuit has voltage V(t) = 120sin(60πt) volts and current I(t) = 5cos(60πt) amperes. The instantaneous power is P(t) = V(t) × I(t).
Check the instantaneous power at time point of 60 secs.

In [7]:
def time_point_verification(timepoint):
  t = torch.tensor(timepoint,dtype=torch.float64, requires_grad= True)
  omega = 60* torch.pi
  V = 120* torch.sin(omega * t)
  I = 5 * torch.cos(omega * t)
  P = V * I #raw equations in textbooks are essentially multiplying and in it's rawest form so U and v can be represented like this

  P.backward()
  d_p = t.grad.item()

  return f"The rate of change power at the time point {timepoint} is {d_p} volts and power is {P.item()}"

cycle =1/60
timepoint = [0,cycle/8,cycle/4,cycle/2]

for t in timepoint:
  print(time_point_verification(t))


The rate of change power at the time point 0 is 113097.33552923254 volts and power is 0.0
The rate of change power at the time point 0.0020833333333333333 is 79971.89288685058 volts and power is 212.13203435596424
The rate of change power at the time point 0.004166666666666667 is 2.1827872842550278e-11 volts and power is 300.0
The rate of change power at the time point 0.008333333333333333 is -113097.33552923254 volts and power is 1.6996616692943938e-13


In [8]:
def power_analysis_jax():
  cycle =1/60
  timepoint = [0,cycle/8,cycle/4,cycle/2]
  omega = 60* torch.pi
  t = jnp.array(timepoint) # had to remind myself here

  def P(t):
    return (120* jnp.sin(omega * t)) * (5 * jnp.cos(omega * t))

  d_p = jax.grad(P)
  result = jax.vmap(d_p)(t)

  print("JAX Results:")
  print("Time | Power")
  print("-" * 40)
  for t in range (len(timepoint)):
      print(f"Time is {timepoint[t]} secs and power is {result[t]} volts")

power_analysis_jax()


JAX Results:
Time | Power
----------------------------------------
Time is 0 secs and power is 113097.3359375 volts
Time is 0.0020833333333333333 secs and power is 79971.8828125 volts
Time is 0.004166666666666667 secs and power is 0.0 volts
Time is 0.008333333333333333 secs and power is -113097.3359375 volts


# Chain rule ( Systems within systems)

The Challenge:
A chemical reactor's temperature depends on pressure: T(P) = 300 + 50√P Kelvin.  
The pressure depends on time: P(t) = 4 + 2sin(0.1t) atmospheres.
How fast is temperature changing at any moment?

In [9]:
def chemical_reactor_temperature_control(time):
  t = torch.tensor(time, dtype = torch.float64, requires_grad = True)

  power = 4 + (2 * torch.sin(0.1 * t))
  T = 300 + (5 * (power ** 0.5))

  T.backward() # reviewed this backward does not go into a variable

  dT_dt = t.grad.item() # was correct but wasn't sure so looked it up.

  return dT_dt
time = [0, 15.7, 31.4, 47.1]
for t in time:
  print(f"Time {t} has temp {chemical_reactor_temperature_control(t)}")


Time 0 has temp 0.25
Time 15.7 has temp 0.00016254951774366278
Time 31.4 has temp -0.24990020166158386
Time 47.1 has temp -0.0008446301065276936


In [10]:
def jax_analysis_chemical_reactor(t):
  time = jnp.array(t)
  def function(time):
    p = 4 + (2* jnp.sin(0.1 * time))
    T = 300 + (5 * jnp.sqrt(p))
    return T

  dT_dt = jax.grad(function)
  result = jax.vmap(dT_dt)(time)

  for i in range(len(t)):
    print(f"Time {t[i]} has temp {result[i]: 6f}")

jax_analysis_chemical_reactor(time)

Time 0 has temp  0.250000
Time 15.7 has temp  0.000163
Time 31.4 has temp -0.249900
Time 47.1 has temp -0.000845


# Optimization : Finding the best solutions
The Challenge:
Design optimal wind turbine blades. Power output follows P(θ) = 1000sin(θ) - 50θ² + 100θ watts, where θ is the blade angle in radians (0 ≤ θ ≤ π/2).

In [12]:
from scipy.optimize import minimize_scalar, fsolve
import numpy as np

def wind_turbine_optimization():
    """
    Wind Turbine Blade Angle Optimization
    -------------------------------------

    Objective:
      Maximize the power output of a wind turbine as a function of blade pitch angle (theta).

    Power Model:
      P(theta) = 1000*sin(theta) - 500*(theta**2) + 100*theta

      - The first term (1000*sin(theta)) models how aerodynamic lift increases with angle up to a point.
      - The second term (-500*theta**2) penalizes excessive pitch angles that reduce efficiency.
      - The third term (+100*theta) captures minor linear contributions (e.g., secondary aerodynamic effects).

    Approach:
      1. Define the power function and its first and second derivatives.
      2. Use `fsolve` to locate critical points (where dP/dθ = 0).
      3. Apply the second derivative test to classify them (maxima/minima).
      4. Compare critical points with boundary points (0 and π/2 radians) to find the global optimum.
    """

    # --- Step 1: Define the objective function ---
    def power_function(theta):
        """Power as a function of blade angle theta (radians)."""
        return 1000*np.sin(theta) - 500*(theta**2) + 100*theta

    # --- Step 2: Define the first derivative dP/dθ ---
    def power_deriv(theta):
        """First derivative of power with respect to theta."""
        return 1000*np.cos(theta) - 1000*theta + 100

    # --- Step 3: Define the second derivative d²P/dθ² ---
    def power_second_deriv(theta):
        """Second derivative of power, used for curvature and classification."""
        return -1000*np.sin(theta) - 1000

    # --- Step 4: Find critical points where dP/dθ = 0 ---
    critical_points = []
    try:
        # Initial guess = 1 rad (~57 degrees)
        root_1 = fsolve(power_deriv, 1)[0]
        # Only consider physically meaningful angles within 0 ≤ θ ≤ π/2
        if 0 <= root_1 <= np.pi/2:
            critical_points.append(root_1)
    except Exception as e:
        print("Root finding failed:", e)

    # --- Step 5: Evaluate power at critical points ---
    for i, cp in enumerate(critical_points):
        power_at_cp = power_function(cp)
        print(f"Critical point {i+1}: θ = {cp:.4f} rad")
        print(f"P(θ_{i+1}) = {power_at_cp:.2f}\n")

    # --- Step 6: Apply the second derivative test ---
    print("Second Derivative Test:\n")
    classified_points = []
    for i, cp in enumerate(critical_points):
        second_deriv = power_second_deriv(cp)
        classification = "Local Maximum" if second_deriv < 0 else "Local Minimum"
        classified_points.append({
            "angle": cp,
            "power": power_function(cp),
            "type": classification
        })

    # --- Step 7: Evaluate boundary points (0 and π/2) ---
    boundary_points = [
        {"angle": 0, "power": power_function(0), "type": "Left Boundary"},
        {"angle": np.pi/2, "power": power_function(np.pi/2), "type": "Right Boundary"}
    ]

    # --- Step 8: Compare all candidates to find global extrema ---
    all_candidates = boundary_points + classified_points
    global_max = max(all_candidates, key=lambda x: x["power"])
    global_min = min(all_candidates, key=lambda x: x["power"])

    # --- Step 9: Display results ---
    print(f"Global Maximum → {global_max['type']} at θ = {global_max['angle']:.4f} rad | Power = {global_max['power']:.2f}")
    print(f"Global Minimum → {global_min['type']} at θ = {global_min['angle']:.4f} rad | Power = {global_min['power']:.2f}")

# Run optimization
wind_turbine_optimization()


Critical point 1: θ = 0.7981 rad
P(θ_1) = 477.36

Second Derivative Test:

Global Maximum → Local Maximum at θ = 0.7981 rad | Power = 477.36
Global Minimum → Right Boundary at θ = 1.5708 rad | Power = -76.62


# Parametric Equations

The Projectile Motion Problem
A soccer player kicks a ball from ground level. The ball's path can be modeled using parametric equations where t represents time in seconds:  
x(t) = 20t
y(t) = 15t - 5t²  
where x is the horizontal distance (in meters) and y is the height (in meters).

Find the Following at Each Time Interval:  
At t = 0 seconds (initial kick):  

 Position (x, y)  
 Velocity dx/dt and dy/dt  
 Slope of trajectory (dy/dx)  
 Angle of trajectory  

At t = 0.5 seconds:  

 Position (x, y)  
 Velocity dx/dt and dy/dt  
 Slope of trajectory (dy/dx)  

At t = 1 second:  

 Position (x, y)  
 Velocity dx/dt and dy/dt  
 Slope of trajectory (dy/dx)  

At t = 1.5 seconds (maximum height):  

 Position (x, y)  
 Velocity dx/dt and dy/dt  
 Slope of trajectory (dy/dx)  
 Verify this is maximum height  

At t = 3 seconds (landing):  

 Position (x, y)  
 Velocity dx/dt and dy/dt  
 Total range traveled  

In [14]:
import torch

def motion_problem(values):
    """
    Simulating Particle Motion Using PyTorch Autograd
    -------------------------------------------------

    Given parametric equations of motion:
        x(t) = 20t
        y(t) = 15t - 5t²

    This function computes:
      - Position (x, y)
      - Velocity components (dx/dt, dy/dt)
      - Slope of the trajectory (dy/dx)
      - Angle of motion (in degrees)

    Args:
        values (list or array): Time values (t) to evaluate.
    """

    # --- Step 1: Create time tensor with autograd enabled ---
    t = torch.tensor(values, dtype=torch.float64, requires_grad=True)

    # --- Step 2: Define motion equations ---
    x = 20 * t
    y = 15 * t - 5 * (t ** 2)

    # --- Step 3: Compute velocity components ---
    d_y = torch.autograd.grad(
        y, t, grad_outputs=torch.ones_like(y), create_graph=True
    )[0]

    d_x = torch.autograd.grad(
        x, t, grad_outputs=torch.ones_like(x), create_graph=True
    )[0]

    # --- Step 4: Compute slope and trajectory angle (radians → degrees) ---
    slope = d_y / d_x
    angle_radians = torch.atan(slope)
    angle_degrees = angle_radians * (180 / torch.pi)

    # --- Step 5: Display results ---
    for i in range(len(values)):
        print(f"\nAt t = {values[i]} s:")
        print(f"  Position (x, y): ({x[i].item():.2f}, {y[i].item():.2f})")
        print(f"  Velocity (dx/dt, dy/dt): ({d_x[i].item():.2f}, {d_y[i].item():.2f})")
        print(f"  Slope of trajectory (dy/dx): {slope[i].item():.2f}")
        print(f"  Angle of motion: {angle_degrees[i].item():.2f}°")

# --- Test Case ---
timepoints = [0, 0.5, 1, 1.5, 3]
motion_problem(timepoints)



At t = 0 s:
  Position (x, y): (0.00, 0.00)
  Velocity (dx/dt, dy/dt): (20.00, 15.00)
  Slope of trajectory (dy/dx): 0.75
  Angle of motion: 36.87°

At t = 0.5 s:
  Position (x, y): (10.00, 6.25)
  Velocity (dx/dt, dy/dt): (20.00, 10.00)
  Slope of trajectory (dy/dx): 0.50
  Angle of motion: 26.57°

At t = 1 s:
  Position (x, y): (20.00, 10.00)
  Velocity (dx/dt, dy/dt): (20.00, 5.00)
  Slope of trajectory (dy/dx): 0.25
  Angle of motion: 14.04°

At t = 1.5 s:
  Position (x, y): (30.00, 11.25)
  Velocity (dx/dt, dy/dt): (20.00, 0.00)
  Slope of trajectory (dy/dx): 0.00
  Angle of motion: 0.00°

At t = 3 s:
  Position (x, y): (60.00, 0.00)
  Velocity (dx/dt, dy/dt): (20.00, -15.00)
  Slope of trajectory (dy/dx): -0.75
  Angle of motion: -36.87°


In [15]:
import jax
import jax.numpy as jnp

def motion_problem_jax(values):
    """
    Simulating Particle Motion Using JAX Automatic Differentiation
    --------------------------------------------------------------
    Given the parametric equations of motion:
        x(t) = 20t
        y(t) = 15t - 5t²

    This function computes:
      - Position (x, y)
      - Velocity components (dx/dt, dy/dt)
      - Slope of trajectory (dy/dx)
      - Instantaneous angle of motion (degrees)

    Args:
        values (list or array): Time values (t) to evaluate.

    Uses:
        JAX automatic differentiation (`jax.grad`)
        + vectorized mapping (`jax.vmap`) for batch evaluation.
    """

    # --- Step 1: Convert time samples into JAX array ---
    t = jnp.array(values)

    # --- Step 2: Define motion equations ---
    def x(t): return 20 * t
    def y(t): return 15 * t - 5 * (t ** 2)

    # --- Step 3: Define symbolic derivatives ---
    # jax.grad constructs differentiable functions of t
    d_x = jax.grad(x)
    d_y = jax.grad(y)

    # --- Step 4: Vectorize derivative evaluation over all t values ---
    dx = jax.vmap(d_x)(t)
    dy = jax.vmap(d_y)(t)

    # --- Step 5: Compute slope and trajectory angle ---
    slope = dy / dx
    angle_rad = jnp.arctan(slope)
    angle_deg = angle_rad * (180 / jnp.pi)

    # --- Step 6: Display results ---
    for i in range(len(values)):
        print(f"\nAt t = {values[i]} s:")
        print(f"  Position (x, y): ({x(t[i]):.2f}, {y(t[i]):.2f})")
        print(f"  Velocity (dx/dt, dy/dt): ({dx[i]:.2f}, {dy[i]:.2f})")
        print(f"  Slope of trajectory (dy/dx): {slope[i]:.2f}")
        print(f"  Angle of motion: {angle_deg[i]:.2f}°")

# --- Test Case ---
timepoints = [0.0, 0.5, 1.0, 1.5, 3.0]
motion_problem_jax(timepoints)



At t = 0.0 s:
  Position (x, y): (0.00, 0.00)
  Velocity (dx/dt, dy/dt): (20.00, 15.00)
  Slope of trajectory (dy/dx): 0.75
  Angle of motion: 36.87°

At t = 0.5 s:
  Position (x, y): (10.00, 6.25)
  Velocity (dx/dt, dy/dt): (20.00, 10.00)
  Slope of trajectory (dy/dx): 0.50
  Angle of motion: 26.57°

At t = 1.0 s:
  Position (x, y): (20.00, 10.00)
  Velocity (dx/dt, dy/dt): (20.00, 5.00)
  Slope of trajectory (dy/dx): 0.25
  Angle of motion: 14.04°

At t = 1.5 s:
  Position (x, y): (30.00, 11.25)
  Velocity (dx/dt, dy/dt): (20.00, 0.00)
  Slope of trajectory (dy/dx): 0.00
  Angle of motion: 0.00°

At t = 3.0 s:
  Position (x, y): (60.00, 0.00)
  Velocity (dx/dt, dy/dt): (20.00, -15.00)
  Slope of trajectory (dy/dx): -0.75
  Angle of motion: -36.87°


# Implicit Differentiation

# 🛰️ Problem 7 – Drone Flight Path Derivative

### Real-World Context
A **drone** is flying in a **perfect circular path** around a **radio tower**.  
The equation describing the drone’s path is:

$$x^2 + y^2 = 25$$

where:
- $x$ and $y$ are the drone’s horizontal coordinates (in meters),
- The tower is at the origin $(0, 0)$,
- The circle has a **radius of 5 m**.

---

### 🎯 Objective
Find the value of:

$$\frac{dy}{dx}$$

when $x = 4$.

---

### 🧩 Interpretation
$\frac{dy}{dx}$ represents the **instantaneous slope** of the drone’s path —  
how steeply its vertical position $y$ changes relative to its horizontal position $x$.  
At $x = 4$, we’ll determine whether the drone is **ascending or descending**,  
and **how sharply** along its circular trajectory.

---

### 🧮 Mathematical Setup
Start with the implicit equation:

$$x^2 + y^2 = 25$$

Differentiate both sides with respect to $x$:

$$2x + 2y\frac{dy}{dx} = 0$$

Solve for $\frac{dy}{dx}$:

$$\frac{dy}{dx} = -\frac{x}{y}$$

At $x = 4$, find $y$ from the circle equation:

$$4^2 + y^2 = 25 \;\Rightarrow\; y = \pm 3$$

Therefore,

$$\frac{dy}{dx} = -\frac{4}{3} \quad \text{(when } y = +3, \text{moving downward)}$$  
$$\frac{dy}{dx} = \frac{4}{3} \quad \text{(when } y = -3, \text{moving upward)}$$

---

### ⚙️ Implementation Plan (for JAX / PyTorch)
We’ll model this using **automatic differentiation**.

1. Define the implicit function  
   $$f(x, y) = x^2 + y^2 − 25 = 0$$
2. Compute partial derivatives  
   $$f_x = \frac{\partial f}{\partial x}, \quad f_y = \frac{\partial f}{\partial y}$$
3. Apply the implicit differentiation formula  
   $$\frac{dy}{dx} = -\,\frac{f_x}{f_y}$$
4. Evaluate this for $(x, y) = (4, 3)$ and $(4, −3)$.

---

### 🚀 Real-World Meaning
This gives the **slope of the drone’s flight direction** at each point:

- When $y = 3$: the drone is **descending**.  
- When $y = −3$: the drone is **ascending**.

You can visualize this by plotting the circular path  
and drawing tangent lines at those two points to show the **direction of motion**.


In [24]:
import torch

def drone_flight_path():
    """
    Drone Flight Path Analysis using PyTorch Autograd
    -------------------------------------------------
    The drone follows a circular path defined implicitly by:
        x² + y² = 25   →   y = sqrt(25 - x²)

    This function computes dy/dx at a given x-position using automatic differentiation.
    The derivative indicates the slope of the drone's trajectory:
        - Negative dy/dx → descending (downward path)
        - Positive dy/dx → ascending (upward path)
    """

    # --- Step 1: Define position along x-axis ---
    x = torch.tensor(4.0, dtype=torch.float64, requires_grad=True)  # Drone at x = 4 m

    # --- Step 2: Define y implicitly from the circle equation ---
    y = torch.sqrt(25 - x**2)  # Circle of radius 5 m

    # --- Step 3: Compute dy/dx using PyTorch autograd ---
    dy_dx = torch.autograd.grad(y, x, create_graph=True)[0]

    # --- Step 4: Interpret result ---
    print(f"At x = {x.item()} m, dy/dx = {dy_dx.item():.3f}")
    print(f"  → The drone is descending at a slope of {dy_dx.item():.3f} m/m.")
    print(f"  → The corresponding ascending slope (on the upper path) is {-dy_dx.item():.3f} m/m.")

# --- Run the simulation ---
drone_flight_path()


At x = 4.0 m, dy/dx = -1.333
  → The drone is descending at a slope of -1.333 m/m.
  → The corresponding ascending slope (on the upper path) is 1.333 m/m.


In [25]:
import jax
import jax.numpy as jnp

def drone_flight_path_jax():
    """
    Drone Flight Path Analysis using JAX Autograd
    ---------------------------------------------
    The drone follows a circular path defined by:
        x² + y² = 25  →  y = sqrt(25 - x²)

    This function computes dy/dx at specific x-positions using JAX's automatic differentiation.
    The derivative tells us the slope of the trajectory:
        - Negative slope → descending motion
        - Positive slope → ascending motion
    """

    # --- Step 1: Define x-coordinate(s) where we analyze the slope ---
    x = jnp.array([4.0])  # Drone position along the x-axis (in meters)

    # --- Step 2: Define y(x) from the implicit circle equation ---
    def y(x):
        return jnp.sqrt(25 - x**2)  # Circle of radius 5 m

    # --- Step 3: Differentiate y(x) with respect to x using JAX autograd ---
    dy_dx_func = jax.grad(y)

    # --- Step 4: Apply vectorized mapping for multiple points (even though we have one here) ---
    dy_dx = jax.vmap(dy_dx_func)(x)

    # --- Step 5: Interpret the result ---
    for i, slope in enumerate(dy_dx):
        print(f"At x = {x[i]:.2f} m:")
        print(f"  dy/dx = {slope:.3f} → Drone is descending at this slope.")
        print(f"  {-slope:.3f} → Corresponds to the mirrored (ascending) path.\n")

# --- Run the simulation ---
drone_flight_path_jax()


At x = 4.00 m:
  dy/dx = -1.333 → Drone is descending at this slope.
  1.333 → Corresponds to the mirrored (ascending) path.

